# Tidy Tensors


In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
import numpy as np

In [ ]:
# | export


class Tensor:
    op = "L"
    name: str = ""
    parents: list = []

    def __init__(self, data, name=""):
        self.data = data
        self.name = name
        self.grad = np.zeros_like(self.data)

    def add(self, other, name):
        out = AddTensor(data=self.data + other.data, name=name)
        out.parents = [self, other]
        return out

    def mul(self, other, name):
        out = MulTensor(data=self.data * other.data, name=name)
        out.parents = [self, other]
        return out

    def mmul(self, other, name):
        out = MulTensor(data=self.data @ other.data, name=name)
        out.parents = [self, other]
        return out

    def sum(self, name):
        out = SumTensor(data=self.data.sum(), name=name)
        out.parents = [self]
        return out

    def backward(self):
        # Create a list of all parent nodes, in reverse order
        # Start with the current node
        visited = []
        nodes = []

        def walk(node):
            for p in node.parents:
                if p not in visited:
                    visited.append(p)
                    walk(p)
                    nodes.append(p)

        walk(self)
        nodes.append(self)

        # print(nodes)
        self.grad = np.ones_like(self.data)
        for n in nodes[::-1]:
            if hasattr(n, "_backward"):
                n._backward()

    def __repr__(self):
        res = f"[{self.op or ''}] {self.name}={str(self.data)} {self.name}.grad={str(self.grad)}"
        if self.parents:
            res += (
                f" {self.name}.parents=[" + ",".join([p.name for p in self.parents]) + "]"
            )

        return f"Tensor({res})"


class AddTensor(Tensor):
    op = "+"

    def _backward(self):
        self.parents[0].grad += self.grad
        self.parents[1].grad += self.grad


class MulTensor(Tensor):
    op = "*"

    def _backward(self):
        self.parents[0].grad += self.grad * self.parents[1].data
        self.parents[1].grad += self.grad * self.parents[0].data


class MmulTensor(Tensor):
    op = "@"

    def _backward(self):
        self.parents[0].grad += self.grad @ self.parents[1].data
        self.parents[1].grad += self.grad @ self.parents[0].data


class SumTensor(Tensor):
    op = "sum"

    def _backward(self):
        self.parents[0].grad += self.grad

In [ ]:
a = Tensor(1, "a")
b = Tensor(2, "b")

c = a.add(b, "c")
d = a.mul(b, "d")
e = c.add(d, "e")


print(e)

e.grad = 1
# e.backward()

Tensor([+] e=5 e.grad=0 e.parents=[c,d])


In [ ]:
a = Tensor(np.ones((10, 10)), "a")
b = Tensor(np.ones((10, 10)) * 2, "b")

c = a.mmul(b, "c")

s = c.sum("s")

s.backward()

In [ ]:
s

Tensor([sum] s=2000.0 s.grad=1.0 s.parents=[c])

In [ ]:
a.grad, b.grad

(array([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]]),
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
#| eval: false
from torch import Tensor, tensor

a = tensor(np.ones((3, 4)), requires_grad=True)
b = tensor(np.ones((4, 3)) * 2, requires_grad=True)

c = (a @ b).sum()

c.backward()
a.grad, b.grad

/home/xl0/mambaforge/envs/torch2/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


(tensor([[6., 6., 6., 6.],
         [6., 6., 6., 6.],
         [6., 6., 6., 6.]], dtype=torch.float64),
 tensor([[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]], dtype=torch.float64))